# FRED-MD Dataset

Paper: https://doi.org/10.1080/07350015.2015.1086655      
Homepage: https://research.stlouisfed.org/econ/mccracken/fred-databases/


Import the dependencies.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Define a function for transforming the time series.

In [2]:
def transform_series(x, tcode):
    '''
    Transform the time series.

    Parameters:
    ______________________________
    x: pandas.Series
        Time series.

    tcode: int.
        Transformation code.
    '''

    if tcode == 1:
        return x
    elif tcode == 2:
        return x.diff()
    elif tcode == 3:
        return x.diff().diff()
    elif tcode == 4:
        return np.log(x)
    elif tcode == 5:
        return np.log(x).diff()
    elif tcode == 6:
        return np.log(x).diff().diff()
    elif tcode == 7:
        return x.pct_change()
    else:
        raise ValueError(f"unknown `tcode` {tcode}")

Define a function for downloading and transforming the time series.

In [3]:
def get_data(year, month, transform=True):

    '''
    Download and (optionally) transform the time series.

    Parameters:
    ______________________________
    year: int
        The year of the dataset vintage.

    month: int.
        The month of the dataset vintage.

    transform: bool.
        Whether the time series should be transformed or not.
    '''

    # get the dataset URL
    file = f"https://files.stlouisfed.org/files/htdocs/fred-md/monthly/{year}-{format(month, '02d')}.csv"

    # download the time series
    data = pd.read_csv(file, skiprows=[1])

    # process the dates
    data["sasdate"] = pd.to_datetime(data["sasdate"], format="%m/%d/%Y")

    # add back any missing months
    data = data.set_index("sasdate").resample("MS").first()

    if transform:

        # load the transformation codes
        tcodes = pd.read_csv(file, nrows=1, index_col=0)

        # transform the time series
        for column in data.columns:
            data[column] = transform_series(x=data[column], tcode=tcodes[column][0])

    return data

Define a function for identifying the time series included in all dataset vintages between two dates.

In [4]:
def get_common_series(start_month, start_year, end_month, end_year):
    '''
    Get the list of time series included in
    all datasets vintages between two dates.

    Parameters:
    ______________________________
    start_month: int.
        The month of the start date.

    start_year: int.
        The year of the start date.

    end_month: int.
        The month of the end date.

    end_year: int.
        The year of the end date.
    '''

    # define the date range
    dates = pd.date_range(
        start=f"{start_year}-{start_month}-01",
        end=f"{end_year}-{end_month}-01",
        freq="MS"
    )

    # get the list of time series included
    # in the dataset on each date
    series = [
        pd.read_csv(
            f"https://files.stlouisfed.org/files/htdocs/fred-md/monthly/{date.year}-{format(date.month, '02d')}.csv",
            nrows=0,
            index_col=0
        ).columns.tolist() for date in dates
    ]

    # return the list of time series included
    # in the dataset on all dates
    return list(set.intersection(*map(set, series)))

Load the dataset vintage for 12-2023.

In [5]:
dataset = get_data(year=2023, month=12, transform=True)

In [6]:
dataset.shape

(779, 127)

In [7]:
dataset.head()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
1959-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-02-01,0.003877,0.003621,0.010349,0.007336,0.007310,0.019391,0.013407,0.008625,0.00731,0.005235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-03-01,0.006457,0.007325,0.009404,-0.003374,0.008321,0.014306,0.006035,0.004894,0.00000,0.019393,...,-0.001148,0.000292,-0.000022,-0.008147,0.004819,NaN,0.004929,0.004138,-0.014792,NaN
1959-04-01,0.006510,0.007029,-0.003622,0.019915,0.000616,0.021075,0.014338,0.014545,0.01565,0.006383,...,0.001312,0.001760,-0.000022,0.012203,-0.004890,NaN,0.012134,0.006734,0.024929,NaN
1959-05-01,0.005796,0.006618,0.012043,0.006839,0.007803,0.014955,0.008270,0.009582,0.00477,0.020149,...,-0.001695,-0.001867,-0.000021,-0.004090,-0.004819,NaN,0.002828,0.002020,-0.015342,NaN


In [8]:
dataset.tail()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
2023-07-01,0.000959,0.002689,0.004674,0.007571,0.005628,0.008918,0.008860,0.011675,0.012764,0.032870,...,-0.000887,0.000308,-0.002738,0.003215,-0.006112,7.3,-0.009362,-0.011050,-0.000322,13.8333
2023-08-01,0.000749,0.001759,-0.000551,-0.001668,0.007353,-0.000008,0.001259,0.001396,0.001537,-0.016683,...,0.014038,-0.001768,-0.000014,0.001425,-0.003026,-2.1,0.001532,0.000716,-0.006009,15.7822
2023-09-01,-0.000509,0.000725,0.003635,0.006531,0.008206,0.001177,-0.002274,-0.004014,-0.002411,0.008238,...,-0.010725,0.003456,0.001660,-0.004090,0.004892,-1.5,-0.003609,-0.001637,0.003319,15.0424
2023-10-01,0.002211,0.003100,0.000754,-0.000689,-0.002209,-0.008928,-0.006722,-0.007656,-0.009127,-0.055520,...,-0.006273,-0.002894,-0.003027,0.001724,-0.004144,-4.1,-0.004214,-0.000859,-0.004454,19.0462
2023-11-01,0.004228,0.005755,0.003200,NaN,0.002759,0.002401,0.001836,0.002633,0.000753,0.034805,...,-0.005675,0.000486,0.005307,0.003420,0.006346,-2.5,NaN,NaN,0.009452,13.8563


In [9]:
dataset.describe()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
count,778.000000,778.000000,778.000000,777.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,...,777.000000,777.000000,777.000000,777.000000,777.000000,550.000000,776.000000,776.000000,777.000000,737.000000
mean,0.002579,0.002413,0.002625,0.002181,0.004699,0.001982,0.001876,0.001944,0.001495,0.002206,...,-0.000012,0.000001,0.000004,0.000005,0.000003,-0.040727,0.000006,0.000003,0.000008,19.367057
std,0.011812,0.006186,0.008002,0.012608,0.015556,0.010040,0.009684,0.010612,0.010662,0.033479,...,0.006143,0.001584,0.003959,0.008914,0.004758,4.026268,0.025514,0.022777,0.010823,7.101231
min,-0.144164,-0.062363,-0.116124,-0.127328,-0.158092,-0.143656,-0.146569,-0.157030,-0.124075,-0.496683,...,-0.037352,-0.009434,-0.029509,-0.068074,-0.026868,-17.300000,-0.195565,-0.259562,-0.084681,9.484400
25%,0.000563,0.000240,-0.000045,-0.003810,-0.001094,-0.001770,-0.002287,-0.002494,-0.003490,-0.008420,...,-0.002614,-0.000850,-0.002220,-0.003384,-0.002460,-2.400000,-0.006881,-0.004007,-0.005889,14.314200
50%,0.002664,0.002805,0.002543,0.002297,0.004881,0.002454,0.002268,0.002165,0.001689,0.002968,...,0.000048,-0.000031,-0.000012,-0.000025,-0.000011,-0.100000,0.000592,0.000594,0.000419,17.660300
75%,0.004721,0.004964,0.005700,0.008438,0.010728,0.006444,0.006455,0.006937,0.006640,0.013340,...,0.002927,0.000787,0.002032,0.003412,0.002198,2.500000,0.006815,0.004885,0.005969,22.560700
max,0.184074,0.040169,0.079478,0.078878,0.174053,0.062986,0.068703,0.082872,0.085326,0.356637,...,0.027667,0.015965,0.019303,0.053050,0.024828,17.300000,0.188590,0.258050,0.049963,62.964800


Get the mapping table with the time series description and groups.

In [10]:
!curl -O https://files.stlouisfed.org/files/htdocs/uploads/FRED-MD%20Appendix.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  236k  100  236k    0     0   518k      0 --:--:-- --:--:-- --:--:--  518k


In [11]:
!unzip -o FRED-MD%20Appendix.zip

Archive:  FRED-MD%20Appendix.zip
  inflating: FRED-MD Appendix/FRED-MD_historic_appendix.csv  
  inflating: FRED-MD Appendix/FRED-MD_historic_appendix.pdf  
  inflating: FRED-MD Appendix/FRED-MD_updated_appendix.csv  
  inflating: FRED-MD Appendix/FRED-MD_updated_appendix.pdf  
  inflating: FRED-MD Appendix/README.txt  


In [12]:
mapping = pd.read_csv("FRED-MD Appendix/FRED-MD_updated_appendix.csv", encoding="ISO 8859-1", usecols=["fred", "description", "group"])

In [13]:
mapping.head()

,fred,description,group
0,RPI,Real Personal Income,1
1,W875RX1,Real personal income ex transfer receipts,1
2,DPCERA3M086SBEA,Real personal consumption expenditures,4
3,CMRMTSPLx,Real Manu. and Trade Industries Sales,4
4,RETAILx,Retail and Food Services Sales,4


In [14]:
mapping.tail()

,fred,description,group
122,UMCSENTx,Consumer Sentiment Index,4
123,DTCOLNVHFNM,Consumer Motor Vehicle Loans Outstanding,5
124,DTCTHFNM,Total Consumer Loans and Leases Outstanding,5
125,INVEST,Securities in Bank Credit at All Commercial Banks,5
126,VIXCLSx,VIX,8


The group mappings are as follows:

In [15]:
groups = {
    1: "Output and Income",
    2: "Labor Market",
    3: "Consumption and Orders",
    4: "Orders and Inventories",
    5: "Money and Credit",
    6: "Interest Rates and Exchange Rates",
    7: "Prices",
    8: "Stock Market"
}

In [16]:
mapping["group"] = mapping["group"].apply(lambda x: groups[x])

In [17]:
mapping.head()

,fred,description,group
0,RPI,Real Personal Income,Output and Income
1,W875RX1,Real personal income ex transfer receipts,Output and Income
2,DPCERA3M086SBEA,Real personal consumption expenditures,Orders and Inventories
3,CMRMTSPLx,Real Manu. and Trade Industries Sales,Orders and Inventories
4,RETAILx,Retail and Food Services Sales,Orders and Inventories


In [18]:
mapping.tail()

,fred,description,group
122,UMCSENTx,Consumer Sentiment Index,Orders and Inventories
123,DTCOLNVHFNM,Consumer Motor Vehicle Loans Outstanding,Money and Credit
124,DTCTHFNM,Total Consumer Loans and Leases Outstanding,Money and Credit
125,INVEST,Securities in Bank Credit at All Commercial Banks,Money and Credit
126,VIXCLSx,VIX,Stock Market


Get the list of time series included in all dataset vintages up to 12-2023.

In [19]:
series = get_common_series(start_month=1, start_year=2015, end_month=12, end_year=2023)

In [20]:
len(series)

118